In [1]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../discharge_me/data/test_phase_2/diagnosis_hadm.csv'
discharge_path ='../discharge_me/data/test_phase_2/discharge.csv'
edstays_path = '../discharge_me/data/test_phase_2/edstays.csv'
radiology_path = '../discharge_me/data/test_phase_2/radiology.csv'
triage_path = '../discharge_me/data/test_phase_2/triage.csv'
target_path = '../discharge_me/data/test_phase_2/discharge_target.csv'
discharge_sections_path = '../discharge_me/data/test_phase_2/discharge_sections.csv'
radiology_sections_path = '../discharge_me/data/test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)



In [2]:
discharge_sections_df.head(4)

,Unnamed: 0,subject_id,hadm_id,CC,Service,Major Surgical Procedure,HPI,PMH,SOC,FH,...,Problem List,Physical Exam,Medication Lists,Pertinent Results,BHC,Transitional Issues,Disposition,Discharge Instructions,Followup Instructions,Discharge Diagnosis
0,0,10001884,24962904,Shortness of Breath,"MEDICINE\n \nAllergies: \nIV Dye, Iodine Conta...",N/A,Ms. ___ is a ___ female with history of \nCOPD...,- COPD/Asthma on home 2L O2\n- Atypical Chest ...,___,Mother with asthma and hypertension. Father wi...,...,,ADMISSION PHYSICAL EXAM:\n====================...,is accurate and complete.\n1. Acetaminophen 32...,ADMISSION LABS: \n=========================\n_...,Ms. ___ is a ___ female with history of \nCOPD...,==========================\n[] For pt's contin...,Extended Care\n \nFacility:\n___\n \n___ Diagn...,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",,PRIMARY:\nCOPD Exacerbation\n\nSECONDARY:\nAfi...
1,1,10003019,22774359,fever,MEDICINE\n \nAllergies: \nRagweed / morphine /...,none,Mr ___ is a ___ with h/o stage IV Hodgkins c1d...,"1. Sarcoidosis, dx skin bx: intestinal & pulmo...",___,"Mother: ___, cardiac disease. \nFather: diver...",...,,ADMISSION EXAM\nVitals: 124/67 on neosynephrin...,The Preadmission Medication list is accurate a...,ADMISSION LABS\n___ 10:40AM BLOOD WBC-0.2* RBC...,___ male with h/o Hodgkin's lymphoma C1D17 ABV...,,Home With Service\n \nFacility:\n___\n \nDisch...,"Dear Mr. ___,\n\nIt has been our pleasure to b...",,"Primary Diagnosis\nNeutropenic Fever, no sourc..."
2,2,10003299,29323205,"left leg weakness, falls",NEUROLOGY\n \nAllergies: \nIodine-Iodine Conta...,None,___ is a ___ RH female with a PMHx of\nparamed...,- prior paramedian pontine infarct (___) \n- r...,___,Mother had stroke in her ___ or ___. Her pate...,...,,Admission Exam:\nVitals: T: 97.4 P: 65 R: 16 ...,The Preadmission Medication list is accurate a...,___ 01:10PM GLUCOSE-125* UREA N-9 CREAT-0.9 ...,___ RH female with a PMHx of paramedian pontin...,,Home With Service\n \nFacility:\n___\n \nDisch...,Dear ___ were hospitalized due to symptoms of ...,,Ischemic stroke
3,3,10003502,20459702,Nausea,MEDICINE\n \nAllergies: \nnifedipine / Amitrip...,None,Ms. ___ is an ___ with atrial fibrillation/flu...,Hypertension/hyperlipidemia\nCoronary artery d...,___,Mother deceased at ___ yo from breast cancer. ...,...,,On admission: \nVS 97.4 140/P 62 18 96\nGEN Al...,The Preadmission Medication list is accurate a...,___ 10:15AM BLOOD WBC-6.4 RBC-3.64* Hgb-11.1* ...,Hospitalization Summary: \nMs. ___ is an ___ y...,- monitoring of volume status and titration of...,Home With Service\n \nFacility:\n___\n \nDisch...,It was a pleasure caring for you at ___ \n___....,,Primary: \nAcute diastolic CHF exacerbation\nN...


In [3]:
row_number = target_df[target_df['hadm_id'] == 25203509].index[0]
print(row_number)

1185


# Load model

In [ ]:
# https://huggingface.co/NouRed/BioMed-Tuned-Llama-3-8b
# Installations
!pip install peft --quiet
!pip install bitsandbytes --quiet
!pip install transformers --quiet
!pip install flash-attn --no-build-isolation --quiet

In [27]:
# Imports
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModelForCausalLM)

# generate_prompt function
def generate_prompt(instruction, input=None):
    if input:
        return f"""

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""

### Instruction:
{instruction}

### Response:
"""

# Model Loading Configuration
based_model_path = "meta-llama/Meta-Llama-3-8B"
lora_weights = "NouRed/BioMed-Tuned-Llama-3-8b"

load_in_4bit=True
bnb_4bit_use_double_quant=True
bnb_4bit_quant_type="nf4"
bnb_4bit_compute_dtype=torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    based_model_path,
    )

tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token = 1024
tokenizer.add_eos_token = True

# Load Base Model in 4 Bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype
)

base_model = AutoModelForCausalLM.from_pretrained(
    based_model_path,
    device_map="auto",
    attn_implementation="flash_attention_2", # I have an A100 GPU with 40GB of RAM 😎
    quantization_config=quantization_config,
    )

# Load Peft Model
peft_model = PeftModel.from_pretrained(
    base_model,
    lora_weights,
    torch_dtype=torch.float16,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# hadm_id = 22774359
# model = peft_model
model = base_model

In [7]:

def generate_with_model(instruction, context_input, model):
    prompt = generate_prompt(instruction, context_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate Text
    with torch.no_grad():
      generation_output = model.generate(
        **inputs,
        max_new_tokens=min(2048, len(inputs)+1024)
      )
    
    # Decode Output
    output = tokenizer.decode(
        generation_output[0], 
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
    
    print(output)
    response = output.split("### Response:")[1].strip()
    return response

# extract relevant information

In [32]:
# extract HPI summary USE THIS
def generate_bhc(hadm_id):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with text and data from a specific patient's medical records and clinical notes
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    Answer clearly and concisely.
    '''.replace("\n", "").replace("\t", "").replace("\r", "").replace("    ", "")
    
    # prompt = '''
    # Using the following patient specific data, 
    # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
    # Then, create a list of the patient's acute and chronic medical conditions and the associated treatments course.
    # '''.replace('\n', '')
    prompt = '''
    Using the following patient-specific medical record text data below delimited by ###,
    summarize the patients relevant past medical history, the reason for hospital admission and the treatment course.
    Focus on major medical conditions and diagnosis, the reason for hospital admission, diagnostic workup, and  the corresponding treatment course.
    Please respond concisely and directly, without using phrases like "based on the information provided" 
    or "according to the text".
    '''.replace("\n", "").replace("\t", "").replace("\r", "").replace("    ", "")
    
    context_str =''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was - {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The History of present illness: \n{context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
    # cols = ['Pertinent Results']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
    cols = ['Major Surgical Procedure']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The major procedures were:\n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"
    
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    context_str = context_str.replace("\n", "").replace("\t", "").replace("\r", "")
    
    prompt = f"{topic} \n: {prompt}"
    context_input = f"<text> Text:{context_str}</text> "


    response = generate_with_model(prompt, context_input, peft_model)
    return response

In [ ]:
# extract medication treatments, med comp use this
def generate_med_comp(hadm_id):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with text and data from a specific patient's medical records and clinical notes
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    Answer clearly and concisely.
    '''.replace('\n', '')
    
    # prompt = '''
    # Given the the following medication list from admission and discharge, 
    # summarize Which medications were new, discontinued, or changed dose and by what amount. 
    # '''.replace('\n', '')
        prompt = '''
    Using the following patient-specific medical record text data delimited by ### 
    summarize Which medications were new, discontinued, or changed dose and by what amount. 
    Please respond concisely and directly, without using phrases like 'based on the information provided' 
    or 'according to the text'.
    '''.replace(["  ", "\t", "\n", "\r"], "")
    
    context_str = ''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['Medication Lists']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The admission medication lists:\n{context_data} \n"
    # context_str = context_str + problem_list
    
    instruction = f"{topic} \n Instruction: {prompt}"
    context_input = f"<text> Text:{context_str}</text> "
    
    response = generate_with_model(instruction, context_input, model)
    
    return response

# generate discharge instructions

In [39]:

def generate_discharge_instructions(hadm_id):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with text and data from a specific patient's medical records and clinical notes
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    Answer clearly and concisely.
    '''.replace("\n", "").replace("\t", "").replace("\r", "").replace("    ", "")
    
    # prompt = '''
    # Given the following information from a patient's hospital stay,
    # Compose a letter to the patient that is courteous and easy to understand. 
    # There should be limited medical jargon and it should be written in layman's language.  
    # The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  
    # It will also summarize any changes to the patient's current medical management including medication changes. 
    # Is there any scheduled or recommended follow up?
    # If there was a surgery, what are the post-operative or wound care instructions?
    # Limit the letter to 3 paragraphs.
    # '''.replace('\n', '')

    prompt = '''
    Using the following patient-specific medical record text data below delimited by ### as context,
    Compose a letter to the patient that is courteous and easy to understand that summarizes their hospital visit. 
    Focus on the the main reason for admission, and the major treatments, diagnostic workup, procedures, and outcomes.
    Use the medication list provided to determine if there are any changes to the medications. Describe any changes to the medications.
    Additionally it will communicate any scheduled or recommended follow up, or other specific post-operative or wound care instructions.
    Please respond concisely and directly, without using phrases like "based on the information provided" 
    or "according to the text". 
    Write the letter in layman's terms using minimal medical jargon. Do not use abbreviations to describe the patient's symptoms.
    Limit your response to 5 paragraphs. 
    '''.replace("\n", "").replace("\t", "").replace("\r", "").replace("    ", "")
    
    context_str = ''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The history of present illness was: {context_data} \n"
    cols = ['Medication Lists']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The admission medication list: \n{context_data} \n"
    cols = ['Transitional Issues']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The ongoing issues to be addressed after discharge: \n{context_data} \n"
    # cols = ['BHC']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
    # instruction = f"{topic} \n Instruction: {prompt}"
    # context_input = f"<text> Text:{context_str}</text> "
    
    context_str = context_str.replace("\n", "").replace("\t", "").replace("\r", "")
    
    prompt = f"{topic} \n: {prompt}"
    context_input = f"<text> Text:{context_str}</text>"
    
    response = generate_with_model(prompt, context_input, base_model)
    return response


In [41]:
len(target_df.hadm_id)

10962

In [42]:
import pandas as pd
# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'brief_hospital_course', 'discharge_instructions'])

# results path:
results_path = '../discharge_me/data/scoring/submission_01/'
file_name = 'results.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(target_df.hadm_id)
for hadm_id in target_df['hadm_id'][:]:  # Replace with your actual hadm_id values
    print(hadm_id)
    count += 1
    print(f"hadm_id {count}: {hadm_id} of {total_records}")
    # brief_hospital_course = generate_bhc(hadm_id)
    brief_hospital_course = ""
    discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'brief_hospital_course': [brief_hospital_course], 'discharge_instructions': [discharge_instructions]})
    results = pd.concat([results, new_row], ignore_index=True)
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


24962904
hadm_id 1: 24962904 of 10962


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
: Using the following patient-specific medical record text data below delimited by ### as context,Compose a letter to the patient that is courteous and easy to understand that summarizes their hospital visit. Focus on the the main reason for admission, and the major treatments, diagnostic workup, procedures, and outcomes.Use the medication list provided to determine if there are any changes to the medications. Describe any changes to the medications.Additionally it will communicate any scheduled or recommended follow up, or other specific post-operative or wound care instructions.Please respond conc

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
: Using the following patient-specific medical record text data below delimited by ### as context,Compose a letter to the patient that is courteous and easy to understand that summarizes their hospital visit. Focus on the the main reason for admission, and the major treatments, diagnostic workup, procedures, and outcomes.Use the medication list provided to determine if there are any changes to the medications. Describe any changes to the medications.Additionally it will communicate any scheduled or recommended follow up, or other specific post-operative or wound care instructions.Please respond conc

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




### Instruction:
The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) from a patient's hospital admission. You will be provided with text and data from a specific patient's medical records and clinical notesdelimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  Answer clearly and concisely. 
: Using the following patient-specific medical record text data below delimited by ### as context,Compose a letter to the patient that is courteous and easy to understand that summarizes their hospital visit. Focus on the the main reason for admission, and the major treatments, diagnostic workup, procedures, and outcomes.Use the medication list provided to determine if there are any changes to the medications. Describe any changes to the medications.Additionally it will communicate any scheduled or recommended follow up, or other specific post-operative or wound care instructions.Please respond conc

KeyboardInterrupt: 

# scoring

In [ ]:
results_file_path = os.path.join(results_path, 'results_bhc.csv')
results_df = pd.read_csv(results_file_path, keep_default_na=False)
results_df.head()


In [33]:
!pip install numpy
!pip install six
!pip install bert_score
!pip install rouge_score
!pip install spacy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: unknown command "installsix" - maybe you meant "install"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 78.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 13.5 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.3 MB/s eta 0:00:0000:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=58c496f44b594993b052d82ac8cf4a04e2b9858cd843de76b5264667bf3fde6d
  Stored in directory: /home/ubuntu/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.3/922.3 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 92.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [40]:
import os
import json
import numpy as np
import pandas as pd

# import evaluate

from ../discharge_me/data/scoring import bleu
# from rouge import Rouge
# from bertscore import BertScore
# from align import AlignScorer
# from UMLSScorer import UMLSScorer

SyntaxError: invalid syntax (2752709126.py, line 8)

In [ ]:
## notes
# input_text = "Your input text here"

# prompt = """
# Summarize the main points from the following text in a single paragraph:
# {}
# Make a list of the main and minor points:
# {}
# Please respond concisely and directly, without using phrases like 'based on the information provided' or 'according to the text'.""".format(input_text, "Points:")

# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(**inputs)
# summary = tokenizer.decode(outputs[0], skip_special_tokens=True)